# Paquetes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers
import os

# Parametros

In [2]:
# MODEL

_LANGUAGE_         = 'es'
_PRETRAINED_LM_    = 'pysentimiento/robertuito-base-cased'
_PREPROCESS_TEXT_  = True
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.UniPELTConfig()
_MAX_SEQ_LEN_      = 128
_OUTPUT_DIR_       = 'adapter_checkPoints_es'
_LOGGING_STEPS_    = 50
_SAVE_DIR_         = 'UniPELT_weights'


# TRAIN

_NO_GPUS_          = 1
_BATCH_SIZE_       = 100
_EPOCHS_           = {'gender': 8, 'variety': 10}
_LEARNING_RATE_    = 1e-4

# Dataset

In [3]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 
                  'male':   1}

varietyEN_dict = {'australia'    : 0,
                  'canada'       : 1,
                  'great britain' : 2,
                  'ireland'      : 3,
                  'new zealand'   : 4,
                  'united states': 5}

varietyES_dict = {'argentina': 0,
                  'chile'    : 1,
                  'colombia' : 2,
                  'mexico'   : 3,
                  'peru'     : 4,
                  'spain'    : 5,
                  'venezuela': 6}    

In [4]:
# SET LANGUAGE DIRECTORY

if _LANGUAGE_ == 'en':
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    variety_dict = varietyES_dict

In [5]:
# SET LANGUAGE TOKENIZER

from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
    
vocab = tokenizer.get_vocab()

In [6]:
# CREATE ONE INSTANCE PER DATA SPLIT

from DatasetPAN17 import BasePAN17, DatasetPAN17

baseTrain = BasePAN17(Dir              = 'data',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)

baseTest  = BasePAN17(Dir              = 'data',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done

Total Instances: 84000


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done

Total Instances: 56000



In [7]:
# CREATE DATA LOADER FOR EVERY TASK

tasks = ['gender', 'variety']

dataset_dict = {}
for task in tasks:
    dataset_dict[task] = DatasetPAN17(Base_Dataset = baseTrain, label = task)

# Model

In [8]:
from transformers import AutoAdapterModel


model = AutoAdapterModel.from_pretrained(_PRETRAINED_LM_)


Some weights of the model checkpoint at pysentimiento/robertuito-base-cased were not used when initializing RobertaAdapterModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at pysentimiento/robertuito-base-cased and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able

# Adapters

In [9]:
num_v           = len(baseTrain.variety_dict)
num_labels_dict = {"gender": 2, "variety": num_v,}

# Add adapters
for task_name in tasks:
    
    model.add_adapter(
        adapter_name = task_name, 
        config       = _ADAPTER_CONFIG_
    )
    
    model.add_classification_head(
        head_name    = task_name,
        num_labels   = num_labels_dict[task_name],
      )

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Training

In [10]:
from Training import train_model_with_adapters

train_model_with_adapters(model         = model,
                          dataset_dict  = dataset_dict,
                          epochs        = _EPOCHS_,
                          batch_size    = _BATCH_SIZE_,
                          no_gpus       = _NO_GPUS_,
                          output_dir    = _OUTPUT_DIR_,
                          logging_steps = _LOGGING_STEPS_,
                          learning_rate = _LEARNING_RATE_)

/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 84000
  Num Epochs = 8
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 1
  Total optimization steps = 6720


Step,Training Loss
50,0.646600
100,0.614100
150,0.603500
200,0.603500
250,0.590400
300,0.583600
350,0.605200
400,0.583600
450,0.577200
500,0.574300


Saving model checkpoint to adapter_checkPoints_es/gender/checkpoint-500
Configuration saved in adapter_checkPoints_es/gender/checkpoint-500/gender/adapter_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-500/variety/adapter_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-500/variety/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-500/variety/head_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-500/variety/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-500

Module weights saved in adapter_checkPoints_es/gender/checkpoint-3500/gender/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-3500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-3500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-3500/variety/adapter_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-3500/variety/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-3500/variety/head_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-3500/variety/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-3500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-3500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-3500/variety/head_config.json
Module weights saved in adapter_c

Module weights saved in adapter_checkPoints_es/gender/checkpoint-6500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-6500/variety/adapter_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-6500/variety/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-6500/variety/head_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-6500/variety/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-6500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-6500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/gender/checkpoint-6500/variety/head_config.json
Module weights saved in adapter_checkPoints_es/gender/checkpoint-6500/variety/pytorch_model_head.bin
Deleting older checkpoint [adapter_checkPoints_es/gender/checkpoint-4000] due to args.save_total_limit


Training complete

Step,Training Loss
50,1.603300
100,1.024400
150,0.954600
200,0.870000
250,0.862300
300,0.811200
350,0.817100
400,0.803200
450,0.791400
500,0.802200


Saving model checkpoint to adapter_checkPoints_es/variety/checkpoint-500
Configuration saved in adapter_checkPoints_es/variety/checkpoint-500/gender/adapter_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-500/variety/adapter_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-500/variety/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-500/variety/head_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-500/variety/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/variety/che

Module weights saved in adapter_checkPoints_es/variety/checkpoint-3500/gender/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-3500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-3500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-3500/variety/adapter_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-3500/variety/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-3500/variety/head_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-3500/variety/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-3500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-3500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-3500/variety/head_config.json
Module weights saved in

Configuration saved in adapter_checkPoints_es/variety/checkpoint-6500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-6500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-6500/variety/adapter_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-6500/variety/pytorch_adapter.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-6500/variety/head_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-6500/variety/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-6500/gender/head_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-6500/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints_es/variety/checkpoint-6500/variety/head_config.json
Module weights saved in adapter_checkPoints_es/variety/checkpoint-6500/variety/pytorch_model_head.bin
Deleting older chec

# Testing

In [11]:
from tqdm import tqdm
import torch
import transformers.adapters.composition as AC  
import numpy as np

def test_model_with_adapters(model, baseTest):
    
    tasks  = ["gender", "variety"]
    labels = ["gender", "variety", "joint"]
    num_labels_dict = {'gender': 2, 'variety': len(baseTest.variety_dict)}
    
    model.set_active_adapters(AC.Parallel(*tasks))
    
    successful_preds = { label: 0 for label in labels }
    
    device = "cuda:0" if torch.cuda.is_available() == True else "cpu"
    
    with torch.no_grad():
        
        for author in tqdm(baseTest.authors):
            # finds all instances of author
            author_idx = [idx for idx in range(len(baseTest.data)) if baseTest.data[idx]['author'] == author]

            # get truth labels with fst instance and initialize scores
            fst      = baseTest.data[author_idx[0]]
            truth    = { task: fst[task]                         for task in tasks }
            scores   = { task: np.zeros( num_labels_dict[task] ) for task in tasks }

            for idx in author_idx:
                # creates case in device
                case = {key: torch.tensor(val[idx]).to(device) for key, val in baseTest.encodings.items()}

                # computes all task predictions in parallel
                preds = []
                
                model.set_active_adapters('gender')
                preds.append( model(**case) )
                
                model.set_active_adapters('variety')
                preds.append( model(**case) )

                # get prediction and accumulate
                for task, pred in zip(tasks, preds):
                    y = torch.nn.functional.softmax(pred['logits'], dim = 1).cpu().numpy()[0]
                    scores[task] += y
            
            good_labels = 0
            for task in tasks:
                if np.argmax( scores[task] ) == truth[task]:
                    good_labels            += 1
                    successful_preds[task] += 1
            
            if good_labels == 2:
                successful_preds['joint'] += 1

    accuracy = { label: val/len(baseTest.authors) for label, val in successful_preds.items() }
    
    return accuracy

In [12]:
#from TestingPAN17 import test_model_with_adapters

accuracy = test_model_with_adapters(model, baseTest)

100%|███████████████████████████████████████████████████████████████████████████████| 2800/2800 [38:32<00:00,  1.21it/s]


In [13]:
accuracy

{'gender': 0.8396428571428571,
 'variety': 0.9467857142857142,
 'joint': 0.8003571428571429}

## Guardar adapter

In [14]:
for task in tasks:
    model.save_adapter(save_directory = _SAVE_DIR_ + '_' + task, adapter_name = task)

Configuration saved in UniPELT_weights_gender/adapter_config.json
Module weights saved in UniPELT_weights_gender/pytorch_adapter.bin
Configuration saved in UniPELT_weights_gender/head_config.json
Module weights saved in UniPELT_weights_gender/pytorch_model_head.bin
Configuration saved in UniPELT_weights_variety/adapter_config.json
Module weights saved in UniPELT_weights_variety/pytorch_adapter.bin
Configuration saved in UniPELT_weights_variety/head_config.json
Module weights saved in UniPELT_weights_variety/pytorch_model_head.bin
